In [76]:
import sqlite3
con = sqlite3.connect("libertykorea.db")
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        date TEXT NOT NULL,
        content TEXT NOT NULL,
        keyword TEXT NOT NULL
        );
    """)
con.commit()

OperationalError: database is locked

In [55]:
import time
from selenium import webdriver
import requests
import re
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

WEBDRIVER_PATH = r'./chromedriver'
LIBERTYKOREA_URL = r'http://www.libertykoreaparty.kr/web/news/briefing/delegateBriefing/mainDelegateBriefingView.do'


def app():
    con = sqlite3.connect("libertykorea.db")
    cur = con.cursor()
    
    
    
    
    driver = webdriver.Chrome(WEBDRIVER_PATH)
    
    
    ### add ###
    
    wait=WebDriverWait(driver,10,0.5,[ElementNotVisibleException])

    
    ###########
    
    
    # set options of webdriver
    # options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    # options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 \
    #                     (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36')
    driver.get(LIBERTYKOREA_URL)
    # base url for crawling articles
    baseUrl = "http://www.libertykoreaparty.kr/web/news/briefing/delegateBriefing/readDelegateBriefingView.do"
    seen = set()

    # number of articles in a page
    length = len(driver.find_elements_by_css_selector("div.board_group.st_list1 > table > tbody tr > td.subject > a"))
    print(length)

    # crawl articles in a page
    # TODO: crawl data moving pages
    # TODO: design db structure, insert crawled data into db
    
    #nextClick = driver.find_element_by_xpath("//div[@id='LblockListTable01']/div[@class='board_paging']/a").get_attribute('onclick')
    #print(nextClick)
    #nextId = str(int(re.search("\d+", nextClick).group())+1)
    #print(nextId)

    
    i = 1
    page=1
    

    while i <= length:
        # click nth article
        

        

        onClick1 = driver.find_element_by_css_selector("div.board_group.st_list1 > table > tbody")
        

        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,
                                        "div.board_group.st_list1 > table > tbody > tr:nth-child({}) > td.subject > a".format(i))))


        onClick = onClick1.find_element_by_css_selector("tr:nth-child({})> td.subject > a".format(i)).get_attribute('onclick')
        
        
        time.sleep(1)
        
        
        bbsId = re.search("SPB_\d+", onClick).group()
        # check duplication
        if bbsId in seen:
            continue
        seen.add(bbsId)

        # parse content
        content = parseContent(baseUrl, {'bbsId': bbsId})
        print(i, "th article:", content["title"])

        # insert content into db
        cur.execute("INSERT INTO table1(title,date,content,keyword)VALUES(?,?,?,?)",[
        content["title"],content["date"],content["body"],content["keyword"]])
        con.commit()
        
        i += 1
        if(i-1==length):
            i=1
            page+=1
            nextPage="//div[@class='board_paging']/a[@onclick='_getList(%s);return false; ']" % page

            max_attemps = 10

            while True:

                nextClick=driver.find_element_by_xpath(nextPage)
                print(nextClick.get_attribute('onclick'))
                if nextClick is not None:
                    nextClick.click()

                    break

                else:

                    time.sleep(0.5)
                    max_attemps -= 1
                    print('attemp --')

                if max_attemps == 0:

                    self.fail("Cannot find element.")

            
    driver.close()



def parseContent(url, param):
    html = requests.request('get', url, params=param)
    dom = BeautifulSoup(html.text, 'lxml')
    return {"title": dom.select_one('td.subject2').text, \
           "body": dom.select_one('#txt_print').text, \
           "date": dom.select_one('#txt_date').text, \
           #"keyword": dom.select_one('div.keyword').text.split(': ')[1].split(', ')}
            "keyword": dom.select_one('div.keyword').text}


if __name__ == '__main__':
    app()

10
1 th article: 경기도청의 아무말 대잔치 [장능인 상근부대변인 논평]
2 th article: 현장 최고위원회의 주요내용[보도자료]
3 th article: 문재인 정권은 일본의 수출규제 조치로 인한 기업 현장의 절규가 들리지도 않는가? [전희경 대변인 논평]
4 th article: 당정청 회의, 대책회의가 아니라 결의 다지기였나 [민경욱 대변인 논평]
5 th article: 조국 교수, 학생들의 바람대로 본인이 좋아하는 앙가주망‘만’ 하시라. [황규환 청년부대변인 논평]
6 th article: 조은누리 양의 회복을 기원하며, 생명을 위한 값진 땀방울을 흘린 군인과 경찰, 소방대원에게 감사를 전한다 [장능인 상근부대변인 논평]
7 th article: 제 자식은 ‘다양화’ ‘자유화’ 누리고 남의 자식에는 ‘일원화’ ‘평준화’ 강요하는 위선교육이 자사고 폐지 본질이다 [전희경 대변인 논평]
8 th article: 다시는 일본에 지지 않겠다는 문재인 대통령, 사태의 조속한 해결을 위한 필사의 노력이 필요하다 [민경욱 대변인 논평]
9 th article: 북핵 때문에 마음 편히 산다는 ‘가짜 진보’ 세력의 자작 테러에 대한 강력한 처벌을 촉구한다! [장능인 상근부대변인 논평]
10 th article: 의원총회 주요내용[보도자료]
_getList(2);return false; 
1 th article: 제11차 북핵외교안보특별위원회 회의 주요내용[보도자료]
2 th article: 일본, 기어이 넘지 말아야 할 선을 넘었다 [민경욱 대변인 논평]
3 th article: 일본수출규제대책특위 긴급회의 주요내용[보도자료]
4 th article: 천덕꾸러기가 된 조국 교수, 페북 할 시간에 학생들의 미래나 걱정하라 [민경욱 대변인 논평]
5 th article: 북한의 세 번째 도발, 한반도가 북한의 무기 성능 시험장이 된 것인가 [민경욱 대변인 논평]
6 th article: 나경원 원내대표, 추경 관련 긴급 기자간담회 주요내용[보도자료]
7 

KeyboardInterrupt: 